To execute succesfully this notebook you should to install the following requirements:

* `langchain==0.1.11`
* `openai==1.13.3`
* `PyPDF2==3.0.1`
* `faiss-cpu==1.8.0`
* `tiktoken==0.6.0`
* `pandas`
* `pydantic==2.6.3`

In [1]:
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [ ]:
# import json

# with open('config.json') as config_file:
#     config = json.load(config_file)
#     open_api_key = config['openai_api_key']

# print(open_api_key)

In [8]:
from dotenv import load_dotenv
import os

load_dotenv()  # Carga las variables de entorno desde .env
open_api_key = os.getenv('OPENAI_API_KEY')

# Simple Call

In [9]:
llm = OpenAI(model_name='gpt-3.5-turbo-instruct', openai_api_key=open_api_key)

In [5]:
our_query = "How many awards did David Palacio won?"
print(llm(our_query))

c:\Users\DAVIDPJ\david\envs\udemy_llm\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




Unfortunately, I could not find any information on a person named David Palacio winning awards. Without more context, I cannot accurately answer this question. 


In [6]:
type(llm)

langchain_community.llms.openai.OpenAI

# LLM Call a PDF

In [7]:
data = PdfReader(r'D:\udemy\LangChain\dpj_description.pdf')

In [8]:
print(len(data.pages))
print(data.metadata)

1
{'/Author': 'XXXXX', '/Creator': 'Microsoft® Word 2019', '/CreationDate': "D:20240307220345-05'00'", '/ModDate': "D:20240307220345-05'00'", '/Producer': 'Microsoft® Word 2019'}


In [28]:
combined_text = ''
for i, page in enumerate(data.pages):
    text = page.extract_text()
    if text:
        combined_text += text
    
print(combined_text)
print("\n")
print(type(combined_text))
print("\n")
print("La cantidad de caracteres que tiene el pdf corresponden a: " + str(len(combined_text)))

David is a Colombian g uy dedicated to be a n AI engineer focu sed on Generative AI models, regression 
models and unbalanced cases associated with geological problems.  
 
David was awarded a scholarship for his Master of Science  degree in Analytics . Also he took sexond place 
in chess tournaments in the department of Antioquia – Colombia.  
 
He is 29 years old.  


<class 'str'>


La cantidad de caracteres que tiene el pdf corresponden a: 370


In [32]:
text = page.extract_text()
text
print(type(text))

<class 'str'>


In [33]:
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 370,
    chunk_overlap = 20,
    length_function = len,
)

finalData = text_splitter.split_text(combined_text)
print(finalData)
print("\n")
print(type(finalData))
print(len(finalData))

['David is a Colombian g uy dedicated to be a n AI engineer focu sed on Generative AI models, regression \nmodels and unbalanced cases associated with geological problems.  \n \nDavid was awarded a scholarship for his Master of Science  degree in Analytics . Also he took sexond place \nin chess tournaments in the department of Antioquia – Colombia.  \n \nHe is 29 years old.']


<class 'list'>
1


In [15]:
#finalData[0]
finalData[1]

'David was awarded a scholarship for his Master of Science  degree in Analytics . Also he took sexond place \nin chess tournaments in the department of Antioquia – Colombia.  \n \nHe is 29 years old.'

## FAISS

Facebook AI Similarity Search (Faiss) is a library for efficient similarity search and clustering of dense vectors. It contains algorithms that search in sets of vectors of any size, up to ones that possibly do not fit in RAM. It also contains supporting code for evaluation and parameter tuning. To see more information click [here](https://python.langchain.com/docs/integrations/vectorstores/faiss).

In [36]:
embeddings = OpenAIEmbeddings(openai_api_key=open_api_key)

In [37]:
documentSearch = FAISS.from_texts(finalData, embeddings)

In [51]:
chain = load_qa_chain(OpenAI(openai_api_key=open_api_key), chain_type="stuff")

In [52]:
our_query = "How old is David Palacio?"
docs = documentSearch.similarity_search(our_query)
chain.run(input_documents=docs, question=our_query)

c:\Users\DAVIDPJ\david\envs\udemy_llm\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


' David Palacio is 29 years old.'